### **Templates - HammingWeight of `w0` 4-bytes**
--------------

##### **Libraries**
--------------

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import scared
import estraces

from IPython.display import display
from ipywidgets import interact

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Useful functions and parameters
plt.rcParams["figure.figsize"] = (20,2)

vhex = np.vectorize(hex)
vhex2 = np.frompyfunc(format, 2, 1) 

#### **Loading the data sets**
--------

In [3]:
ths_building = scared.traces.read_ths_from_ets_file('./dataset/profiling_hw_w0_v4.zip.ets')
ths_matching = scared.traces.read_ths_from_ets_file('./dataset/matching/matching_fixed_v3.zip.ets')

#### **Loading POIs**
--------

In [4]:
pois = np.load('./dataset/pois_profiling_hw_w0_v4.npz')['pois']
print(pois)

[[113 114 118 121 117]
 [113 114 118 121 117]
 [ 18  90  17  81  89]
 [ 82  89  18  17  81]]


#### **Template  definition**
-----------
We build 4 templates, one for each of the `w0` words with:
- The corresponding POIs found during the leakage detection
- The same selection function (identity of `w0`) used for the POIs selection
- The HammingWeight model applied to each of the `w0` word

In [5]:
def identity_w0(msg, w0):
    return w0

S = scared.reverse_selection_function(function = identity_w0)

In [6]:
TAttacks = [scared.TemplateAttack(container_building = scared.Container(ths_building, frame = pois[word]),
                                  reverse_selection_function = scared.reverse_selection_function(function = identity_w0, words = word),
                                  model = scared.HammingWeight(),
                                  convergence_step = 1,
                                  partitions = np.arange(9)) for word in range(0, 4)]

#### **Build the templates**
-----------

In [8]:
for T in TAttacks:
    T.build()

In [9]:
@interact(word = (0, 3, 1))
def widget_template(word=0):
    templates = TAttacks[word].templates
    plt.plot(templates.T)
    plt.title(f"Template means for word {word}")
    plt.show()

interactive(children=(IntSlider(value=0, description='word', max=3), Output()), _dom_classes=('widget-interact…

#### **Matching** 
-----------
We now perform the matching phase with 10 traces of the matching data set

In [10]:
ths_matching = ths_matching[:10]

In [11]:
# Get the secret value to recover
ref = ths_matching.w0[0]
vhex(ref)

array(['0x0', '0x0', '0x0', '0x0'], dtype='<U3')

In [12]:
for word in range(4):
    TAttacks[word].run (scared.Container(ths_matching[:10], frame = pois[word]))

K = np.array([T.scores.argmax(0).squeeze() for T in TAttacks], 'uint8')

if np.array_equal(K, ref):
    print("The secret key is found: ", vhex(K))
else:
    print("The recovered key is: ", vhex(K))
    print("The secret key is:    ", vhex(ref))

The secret key is found:  ['0x0' '0x0' '0x0' '0x0']


In [13]:
@interact(word = (0, 3, 1))
def InteractivePlotTemplateResults(word=3):   
    # Plot the scores
    T = TAttacks[word]
    s = T.scores.squeeze()   
    
    plt.subplot(121)
    plt.bar(range(len(s)), s)
    plt.bar(ref[word], s[ref[word]]) # Plot in orange the expected value
    plt.ylim((np.min(s), np.max(s)))
    plt.xlabel("Key guesses")
    plt.ylabel("Scores")
    plt.title("Templates Scores: Byte %d"%(word))

    # Visualize scores increasing with the number of matching traces
    x = np.arange(0, len(ths_matching), T.convergence_step) + T.convergence_step

    c = T.convergence_traces.squeeze()
    c = c - c.mean(0)

    plt.subplot(122)
    plt.plot(x, c.T, 'C0')
    plt.plot(x, c[ref[word]], 'C1')
    plt.xlabel("Number of traces")
    plt.ylabel("Scores")
    plt.title("Convergence: Byte %d"%(word))
    
    plt.show()

interactive(children=(IntSlider(value=3, description='word', max=3), Output()), _dom_classes=('widget-interact…